## Fine Tuning TinyLlama

Here, the purpose is to make teach the model the common words in and the style of Taylor Swift's song lyrics.

So there is no separate instruction here in the dataset.

Data - https://github.com/shaynak/taylor-swift-lyrics/blob/main/songs.csv

In [ ]:
!pip install trl transformers accelerate peft bitsandbytes einops wandb -Uqqq

In [ ]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset
import glob


## Load all .csv files from a directory which contains the Taylor Swift Lyrics

In [ ]:
import logging

def find_csv_files(path, file_extension="*.csv"):
    """
    Find all CSV files in the given directory.

    :param path: str, the directory to search in
    :param file_extension: str, pattern to match files
    :return: list of file paths
    """
    try:
        files = glob.glob(f"{path}/{file_extension}")
        if not files:
            logging.warning(f"No files found in {path} with extension {file_extension}")
        return files
    except Exception as e:
        logging.error(f"Error finding files in {path}: {e}")
        return []

def read_csv_files(file_paths, column_name='Lyrics'):
    """
    Read and concatenate CSV files into a single DataFrame.

    :param file_paths: list of file paths
    :param column_name: str, name of the column to extract
    :return: DataFrame
    """
    df_list = []
    for file in file_paths:
        try:
            df = pd.read_csv(file)
            if column_name in df.columns:
                df_list.append(df)
            else:
                logging.warning(f"Column {column_name} not found in {file}")
        except Exception as e:
            logging.error(f"Error reading {file}: {e}")
    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()

def concatenate_lyrics(df, column_name='Lyrics'):
    """
    Concatenate lyrics from a DataFrame into a single string.

    :param df: DataFrame containing the lyrics
    :param column_name: str, the column containing lyrics
    :return: str, concatenated lyrics
    """
    if column_name in df.columns:
        return '\n'.join(df[column_name])
    logging.warning(f"Column {column_name} not found in DataFrame")
    return ""

def load_and_concatenate_lyrics(path, file_extension="*.csv", column_name='Lyrics'):
    """
    Load CSV files from the specified path, concatenate them,
    and return the combined lyrics.

    :param path: str, the directory containing the CSV files
    :param file_extension: str, pattern to match files
    :param column_name: str, name of the column to extract
    :return: str, concatenated lyrics from all CSV files
    """
    files = find_csv_files(path, file_extension)
    if not files:
        return ""

    df = read_csv_files(files, column_name)
    if df.empty:
        return ""

    return concatenate_lyrics(df, column_name)

path = '../data_input/' # For local machine

# path = '/content/'        # From Colab

lyrics = load_and_concatenate_lyrics(path)

print(lyrics[:200])  # Printing first 200 characters of the lyrics

In [ ]:
# List of all unique characters
# This is just to verify the contents of the lyrics
print(' '.join(sorted(set(lyrics))))

In [ ]:
import re

def replace_characters(text, replacement_dict):
    """Replace characters in a string based on a dictionary mapping."""
    return text.translate(str.maketrans(replacement_dict))

""" See more detailed notes on this replace_characters() method at the
end of this notebook """

def remove_patterns(text, pattern_list):
    """Remove all instances of patterns in a list from a string."""
    for pattern in pattern_list:
        text = re.sub(pattern, '', text)
    return text

def clean_lyrics(lyrics):
    """Clean lyrics by replacing and removing specific characters and patterns."""
    replace_with_space = ['\u2005', '\u200b', '\u205f', '\xa0', '-']
    replace_letters = {'í':'i', 'é':'e', 'ï':'i', 'ó':'o', ';':',', '‘':'\'', '’':'\'', ':':',', 'е':'e'}
    remove_list = ['\)', '\(', '–','"','”', '"', '\[.*\]', '.*\|.*', '—']

    # Replace specific letters
    lyrics = replace_characters(lyrics, replace_letters)

    # Replace specific strings with a space
    for string in replace_with_space:
        lyrics = lyrics.replace(string, ' ')

    # Remove unwanted patterns
    lyrics = remove_patterns(lyrics, remove_list)

    return lyrics

# Example usage
cleaned_lyrics = clean_lyrics(lyrics)



In [ ]:
# This is just to verify the cleaned text
print(''.join(sorted(set(cleaned_lyrics))))


In [ ]:
cleaned_lyrics

In [ ]:
from datasets import Dataset

def create_train_test_datasets(cleaned_lyrics, train_ratio=0.95, segment_length=500):
    """
    Splits the cleaned lyrics into training and testing datasets.
    The training data is further segmented into smaller chunks.

    Args:
    cleaned_lyrics (list): The preprocessed lyrics data.
    train_ratio (float, optional): Ratio of data to be used for training. Default is 0.95.
    segment_length (int, optional): Length of each segment in the training data. Default is 500.

    Returns:
    Tuple[Dataset, list]: A tuple containing the training dataset and the test dataset.
    """
    split_point = int(len(cleaned_lyrics) * train_ratio)
    train_data = cleaned_lyrics[:split_point]
    test_data = cleaned_lyrics[split_point:]

    # Segmenting the training data
    train_data_segments = [train_data[i:i + segment_length]
                           for i in range(0, len(train_data), segment_length)]
    # In above using `range(start, end, step)` syntax
    train_dataset = Dataset.from_dict({'text': train_data_segments})

    return train_dataset, test_data

# Example usage
train_dataset, test_data = create_train_test_datasets(cleaned_lyrics)


# 557

In [ ]:
print(len(train_dataset))

In [ ]:
# Check the dataset structure
# Should output below
"""
Dataset({
    features: ['text'],
    num_rows: 557
})
"""
train_dataset


In [ ]:
from pprint import pprint
pprint(train_dataset[0])

"""
{'text': "Knew he was a killer first time that I saw him\nWondered how many girls he had loved and left haunted\nBut if he's a ghost, then I can be a phantom\nHoldin' him for ransom, some\nSome boys are tryin' too hard, he don't try at all though\nYounger than my exes, but he act like such a man, so\nI see nothing better, I keep him forever\nLike a vendetta ta\nI, I, I see how this is gon' go\nTouch me and you'll never be alone\nI Island breeze and lights down low\nNo one has to know\nIn the middle of the night, in m"} """

In [ ]:
def load_quantized_model(model_identifier: str, compute_dtype: torch.dtype) -> AutoModelForCausalLM:
    """
    Loads a quantized model using the BitsAndBytesConfig with specified parameters.

    Args:
    - model_identifier (str): The name of the model to be loaded.
    - compute_dtype (torch.dtype): The data type for computation in the quantized model.

    Returns:
    - AutoModelForCausalLM: The loaded quantized model.
    """
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=compute_dtype,
    )

    model =  AutoModelForCausalLM.from_pretrained(
        model_identifier,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
    )
    return model

# Load the model
# model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = load_quantized_model(model_name, torch.bfloat16)

In [ ]:
# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [29]:
# Generating lyrics with the base model. The repetition penalty in the generation config prevents the model from continually repeating the same string.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def generate_lyrics(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=250, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    print('INPUT\n', query, '\n\nOUTPUT\n', text_output[len(query):])

generate_lyrics(test_data[200:700], model)


In [ ]:
# Setting arguments for low-rank adaptation

model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05
lora_rank = 32

lora_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, lora_config)

In [ ]:
# Setting training arguments

output_dir = "hf-username/tinylama_taylor_swift" # Model repo on your hugging face account where you want to save your model
per_device_train_batch_size = 3
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_strategy="steps"
save_steps = 10
logging_steps = 10
learning_rate = 2e-3
max_grad_norm = 0.3 # Sets limit for gradient clipping
max_steps = 200     # Number of training steps
warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine" # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    # push_to_hub=True,
    report_to='none'
)

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    max_seq_length=500,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

In [ ]:
trainer.train()

###############################################
## Explanatory Notes
###############################################

```py
def replace_characters(text, replacement_dict):
    """Replace characters in a string based on a dictionary mapping."""
    return text.translate(str.maketrans(replacement_dict))

```


- `str.maketrans(replacement_dict)`: This function creates a translation table. The `replacement_dict` is a dictionary where each key-value pair represents a character to be replaced and its corresponding replacement character. 

📌 This translation table maps Unicode ordinals (or character code points) of the keys to ordinals of the values. The `str.maketrans` function is typically used to create a mapping table.

- `text.translate(translation_table)`: This method returns a copy of `text` where each character has been mapped through the translation table. Characters mapped to `None` are deleted. In this context, it applies the character replacements defined in `replacement_dict` to the string `text`.

In essence, this combination is a highly efficient way to perform multiple character replacements in a string in a single pass, making it more suitable for tasks like cleaning or formatting large text data, compared to using multiple `replace` calls.

-----------------------

Simple Example

```py

dict = {"a": "123", "b": "456", "c": "789"}
string = "abc"
print(string.maketrans(dict))
# {97: '123', 98: '456', 99: '789'}

```


📌 The `maketrans` method when used with a single argument, such as a dictionary, the keys must be single characters and the values are their corresponding replacements.

📌 In above, `string.maketrans(dict)` creates a translation table from the dictionary `dict`. The dictionary keys ('a', 'b', 'c') are single characters, and their ASCII values are 97, 98, and 99 respectively.

📌 The translation table is a dictionary where each key is the ASCII value of the single-character keys from `dict`, and the values are the corresponding values from `dict`.

📌 Thus, the output `{97: '123', 98: '456', 99: '789'}` represents a mapping where 'a' (ASCII 97) is replaced by '123', 'b' (ASCII 98) by '456', and 'c' (ASCII 99) by '789'. This table is used for translating or replacing characters in a string using the `translate` method.

-------

📌 The `maketrans` function must map characters to their replacements, and in Python, characters are often managed internally using their ASCII values. This representation allows for efficient storage and manipulation of characters in memory.

📌 When you pass a dictionary to `maketrans`, it converts the keys from characters to their ASCII values because the **string translation process in Python operates at the byte level**. Using ASCII values directly is more efficient for this process.

📌 Therefore, in your code `string.maketrans(dict)`, the dictionary keys 'a', 'b', and 'c' are automatically converted to their ASCII values 97, 98, and 99, respectively. This conversion forms a mapping table where each ASCII value (representing a character in the original string) is mapped to the corresponding replacement string as defined in your dictionary.

📌 This translation table is then used by the `translate` method of string objects to replace characters in the string based on the provided mappings. For example, if you use this table to translate a string, every occurrence of 'a' will be replaced by '123', 'b' by '456', and 'c' by '789'.

#######################################################3

## Explaining the below block

```py
train_data_segments = [train_data[i:i + segment_length]
                           for i in range(0, len(train_data), segment_length)]

```

📌 The line `for i in range(0, len(train_data), segment_length)` in the list comprehension serves a specific purpose in segmenting the training data. 

This line iterates over the `train_data` list in steps of `segment_length`. For each iteration, it starts at index `i` and ends at `i + segment_length`, creating a new segment of the training data. By specifying `segment_length` as the step size in the `range` function, it ensures that each segment is of the specified length, except possibly the last segment which might be shorter if the total length of `train_data` is not a multiple of `segment_length`.

The usage of `range(start, end, step)` here is an efficient way to iterate through the `train_data` while creating segments of a defined length, which is crucial for managing memory and computational efficiency.